In [3]:
import re

In [47]:
filename = "vehiclecarsample"
infile_path = filename + ".txt" #path relative to the home directory
outfile_path = filename + "_output.txt"
#multiplecommenttest.txt checking on /* parsing
#nextline_test.txt a test with a sample text file with three "/n" gives the output as just one line filename
#comment test.txt tests for counting of class when in a comment
#inheritedclasses.txt has an inherited class, checks on this count
#simplemain.txt
#purecomment.txt
#vehiclecarsample.txt
#inheritedclasses_singleline.txt 

The file(text) is first split on the basis of spaces, curly braces, parentheses and comments of both variety. abstract, final, static are all defined before class therfore the assumption that after class you will have the class name is true. inheritance must always have the "extends" keyword, therefore we count that. comments are a bit tricky, since nested comments don't exist
// /* will be treated as a single line comment. /* // will be treated as a multiline comment.  An active comment activates a comment flag, which then ignores the count of anything.  A comment with a "class" keyword won't count that.  Same with a comment within a comment and other keywords within the comment.  Constructors are counted by the same class name iff the curly brace count is positive.  Note that if the curly brace count ever becomes negative we can expect wrong results.  I.e. suppose you put a "}" and then write the code normally, curly brace count will be negative so we cant recognize classes.  A modification would be to reset the curly_brace count to be zero each time we go into a class, however, that would violate the condition of keeping track of every curly brace.  This superdependence on curly braces isn't a problem, since it is expected that curly braces hold, or we just throw an error.  

In [55]:
line_count = 0
class_count = 0
object_count = 0
comment_count = 0
inherit_count = 0
comment_flag = False
comment_flag_a = False      #this is for // comment
comment_flag_b = False      #this is for /* comment
class_name_flag = False
in_class = False
cbrace_count = 0
parentheses_count = 0
class_word = "$"
constructor_count = 0
stack_flag = False
class_flag = False
class_line_count = 0
comment_flagg = False
comment_line_count = 0
inherit_flag = False
inherit_line_count = 0
object_flag = False
object_line_count = 0
constructor_flag = False
constructor_line_count = 0
symbol_dict = {}
class_stack = []
class_stack_c = []
#nested classes handled with stack structure
#the in conditions that you used earlier still hold, since "/*" is in "/*"
# If we define Parent class constructor inside Child class it will give compile time error for return type and consider it a method.
# But for print method it does not give any compile time error and consider it a overriding method.
# using the above gem we define the class stack
class_flag = False
comment_flagg = False
inherit_flag = False
object_flag = False
constructor_flag = False
class_line_flag = False
access_modifiers = ["public", "private", "protected", "default"]
non_access_modifiers = ["final", "static", "abstract", "strictfp", "native", "synchronized", "transient", "volatile"]
cbc = 0
icbc = 0

f = open(infile_path, 'r')
stream = f.read()

col_split = re.split(';|[{]|[}]', stream)
#print("col_split", col_split)
split_col_split = []
for elem in col_split:
    temp = re.split("(//.*[\n]|/[*]|[*]/|[\n]|[.]| +)", elem) #everything except colons and curly braces, cuts out comments too
    split_col_split.append(temp)
#print(split_col_split[0])
count = 0
new_count = 0

nothing_flag = True

class_signature_count = 0
inherit_signature_count = 0

for elem in split_col_split:
    if "new" in elem:     #counts object declarations lines
        new_count = new_count + 1
        #print("elem that you want")
        #print(elem)
        nothing_flag = True
        for e in elem:
            if (e == '' or " " in e or "//" in e or "/*" in e or "*/" in e):
                if nothing_flag == True:
                    nothing_flag = True
            else:
                if e == '\n':
                    if nothing_flag == False:
                        count = count + 1
                        #print("incremented at ", e)
                        nothing_flag = True
                else:
                    nothing_flag = False
                    #print("false set at", e)
    if "class" in elem:      #counts class declarations lines
        #print("class elem that you wnat", elem)
        cf_n = False
        for e in elem:
            if e == "class":
                break
            if e in access_modifiers or e in non_access_modifiers:
                cf_n = True
            if cf_n == True and e == '\n':
                class_signature_count = class_signature_count + 1
    if "extends" in elem:   #counts inherited class lines
        #print("inherit elem that you want", elem)
        cf_n = False
        for e in elem:
            if e == "extends":
                break
            if e in access_modifiers:
                cf_n = True
            if cf_n == True and e == '\n':
                inherit_signature_count = inherit_signature_count + 1
print("class singature count ",class_signature_count)
print("inherit singature count ",inherit_signature_count)
print("object signature count ", count)
print("new count (object): ", new_count)
f.close()
f = open(infile_path, 'r')
for line in f:
    #words = line.split()
    words = re.split("([*]/|/[*]|//| +|[{]|[}]|[(]|[)]|[\n])", line)
    line_count  = line_count + 1
    comment_flag_a = False   #reset for every newline
    comment_flag = comment_flag_a or comment_flag_b
    #print(words)
    for word in words:
        #print(word)
        #print(class_stack)
        #print(class_stack_c)
        #print(class_word)
        if word == "" or " " in word or word == "\n": 
            continue;
            ##this works because it bypasses the spaces, note that the space output can be multiline
            #eg : ['', '          ', '\n']
            #'' means that two delimiters match side by side and that nothing resides between them
        if class_name_flag:
            if word not in symbol_dict:
                symbol_dict[word] = 0 #counter counts instances of constructors
                class_word = word  #makes sure that a "constructor" in another class will not increment the current count
                stack_flag = True  #by using the stack flag
            else:
                print("There is already a class with this name, Classname error")
            class_name_flag = False  # we can take advantage of the class_name_flag
            continue
        if comment_flag:
            if comment_flag_a:
                continue
            if comment_flag_b:
                if "*/" in word:
                    comment_flag_b = False
                    comment_count = comment_count + 1
                    comment_flag = False
        if not comment_flag: #each word is one of the following
            if "//" in word:  #no need to handle the case of //akhil being a word, since that is taken care by the split function
                comment_flag_a = True
                comment_count = comment_count + 1
                comment_flag = True
            elif "/*" in word:
                comment_flag_b = True 
                comment_flag = True
            elif word == "class":
                class_count = class_count + 1
                class_name_flag = True
                in_class = True #ideally should be true when we reach the curly braces, not properly defined
                class_flag = True
            elif word == "new":
                object_count = object_count + 1
                #object_flag = True
            elif word == "extends":  #handles parent child constructor definitions, super fails in this context, since we don't define a 
                inherit_count = inherit_count + 1       #we dont count implicit constructor definitions
                inherit_flag = True
            else:
                #we expect the curly braces to be where the stack elements are added
                #we also expect the curly braces to be immediate after the class name
                if '{' in word:
                    cbrace_count = cbrace_count + 1
                    if stack_flag == True: #true only if you have a new class name
                        class_stack.append(class_word)
                        class_stack_c.append(1)
                        stack_flag = False
                    else:
                        #assumed for the most recent class
                        temp = class_stack_c.pop()
                        temp = temp + 1
                        class_stack_c.append(temp)
                if '}' in word:
                    cbrace_count = cbrace_count - 1  #dont worry about in vs ==
                    #a constructor function in a different class?
                    #the inclass variable might be useless
                    temp = class_stack_c.pop()
                    temp = temp - 1
                    if (temp == 0): #we are out of the current class
                        #print(line_count, class_word)
                        if len(class_stack) > 0:
                            class_word = class_stack.pop()
                            #class_flag = False
                            #class_line_count = class_line_count + 1 #
                            class_line_flag = True
                        else:
                            class_word = "$"
                            #class_flag = False  we dont set it false
                            #class_line_count = class_line_count + 1 #factors in current line
                            class_line_flag = True
                        print("out of class here", line_count, class_word)
                    else:
                        class_stack_c.append(temp)
                    if cbrace_count < 0:
                        print("{} braces don't match")
                if '(' in word:
                    parentheses_count = parentheses_count + 1
                if ')' in word:
                    parentheses_count = parentheses_count - 1
                    if parentheses_count < 0:
                        print("() braces don't match")
                temp_count = 0
                if (len(class_stack_c) > 0):
                    temp_count = class_stack_c[-1]
                if word in symbol_dict and word == class_word and temp_count == 1:  #checks for constructors, temp_count is related to the constructor
                    print("Constructor found for", word)
                #temp_count should be 1 for it to be 1
                    constructor_flag = True
                    symbol_dict[word] = symbol_dict[word] + 1  #word is current word, class_word is the name of the current class
                    counstructor_count = constructor_count + 1
                if constructor_flag:
                    if word == '{':
                        cbc = cbc + 1
                    elif word == '}': 
                        cbc = cbc - 1
                        if cbc == 0: #we are out of the constructor
                            constructor_line_count = constructor_line_count + 1 #counts the last line of the constructor
                            constructor_flag = False
                            cbc = 0
                if inherit_flag:
                    if word == '{':
                        icbc = icbc + 1
                    elif word == '}':
                        icbc = icbc - 1
                        if icbc == 0:  #we are out of the inherit class
                            inherit_line_count = inherit_line_count + 1
                            inherit_flag = False
        #dont worry about curly braces outside a class that's impossible
        comment_flag = comment_flag_a or comment_flag_b
    #print("class flag", class_flag, line_count)
    
#     if class_line_flag:
#         class_line_count = class_line_count + 1
#         class_line_flag = False
#         print("class_line_flag high at", line)
    if class_flag:
        class_line_count = class_line_count + 1
        if (class_line_flag):
            class_flag = False
            class_line_flag = False
    if comment_flagg:
        comment_line_count = comment_line_count + 1
    if inherit_flag:
        inherit_line_count = inherit_line_count + 1
    if object_flag:
        object_line_count = object_line_count + 1
    if constructor_flag:
        constructor_line_count = constructor_line_count + 1
    #print("class_line_count", class_line_count, line_count)
#end of the parsing
for i in symbol_dict:  #symbol_dict actually contains the different classes and the count is number of constructors
    constructor_count = constructor_count + symbol_dict[i]
# stream = f.read()
# print(stream)
o = open(outfile_path, 'w')

# print("number of lines contributing to the different instances:\n")
#new_count is the same as object_count
#new_count is number of new instances
#count is number of /n in the new definition
#print("object count is ", object_count)
object_count1 = count + new_count
class_line_count1 = class_line_count + class_signature_count
inherit_line_count1 = inherit_line_count + inherit_signature_count
#updated counts to handle changes in signature spacing

# print("number of such instances:\n")
# print("1) Objects declaration:         ", object_count)
# print("2) Class definition:            " , class_count)
# print("3) Constructor definition:      ", constructor_count)
# print("4) Inherited Class definition:  ", inherit_count)
# print("5) Number of lines:             ", line_count)
# print("6) Comment Count:               ", comment_count)
# print("\n")

print("1) Objects declaration:         ", object_count1)
print("2) Class definition:            " , class_line_count1)
print("3) Constructor definition:      ", constructor_line_count)
print("4) Inherited Class definition:  ", inherit_line_count1)
print("5) Number of lines:             ", line_count)
print("6) Comment Count:               ", comment_count) 
print("Only first four lines written into file")
o.write("1) Objects declaration:         " + str(object_count1) + '\n')
o.write("2) Class definition:            " + str(class_line_count1) + '\n')
o.write("3) Constructor definition:      " + str(constructor_line_count) + '\n')
o.write("4) Inherited Class definition:  " + str(inherit_line_count1) + '\n')
# o.write("5) Number of lines:             " + str(line_count) + '\n')
print(symbol_dict)
# print(cbrace_count)
# print(parentheses_count)
f.close()
o.close()
#expected counts for sample1
# line_count:     81
# class_count:    3
# object_count:   1
# comment_count:  15
# inherit_count:  1
# {'Bicycle': 1, 'MountainBike': 1, 'Test': 0}
# 0
#after making comment change
# line_count:     81
# class_count:    3
# object_count:   1
# comment_count:  18
# inherit_count:  1
# {'Bicycle': 1, 'MountainBike': 1, 'Test': 0}
#how do you get constructor count? another pass will fail because
#bicycle should have a constructor
#inheritedclasses.txt
# line_count:     25
# class_count:    2
# object_count:   2
# comment_count:  2
# inherit_count:  1
# {'Parent': 1, 'Child': 0}
#nestedclasses.txt before nestedconstructor
# line_count:     46
# class_count:    3
# object_count:   1
# comment_count:  14
# inherit_count:  0
# {'OuterClass': 0, 'StaticNestedClass': 0, 'StaticNestedClassDemo': 0}
# 0
#nestedclasses.txt after nestedconstructor
# StaticNestedClass
# line_count:     47
# class_count:    3
# object_count:   1
# comment_count:  14
# inherit_count:  0
# {'OuterClass': 0, 'StaticNestedClass': 1, 'StaticNestedClassDemo': 0}
# 0

class singature count  0
inherit singature count  0
object signature count  0
new count (object):  1
out of class here 6 Vehicle
Constructor found for Car
out of class here 22 Car
1) Objects declaration:          1
2) Class definition:             21
3) Constructor definition:       1
4) Inherited Class definition:   15
5) Number of lines:              22
6) Comment Count:                6
Only first four lines written into file
{'Vehicle': 0, 'Car': 1}
